In [2]:
import cv2
import numpy

In [18]:
def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    try:
        output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    except:
        output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers


def draw_prediction(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])

    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)


def label_frame(image):
    
    Width = image.shape[1]
    Height = image.shape[0]
    scale = 0.00392

    config = "yolov3.cfg"
    weights = "yolov3.weights"
    classes_input = "yolov3.txt"
    net = cv2.dnn.readNet(weights, config)
    COLORS = np.random.uniform(0, 255, size=(len(classes), 3))
    
    blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

    net.setInput(blob)

    outs = net.forward(get_output_layers(net))

    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.75
    nms_threshold = 0.4


    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])


    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    img_to_ret = image.copy()
    
    for i in indices:
        try:
            box = boxes[i]
        except:
            i = i[0]
            box = boxes[i]

        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        draw_prediction(img_to_ret, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))
    
    return img_to_ret

In [19]:
test = cv2.imread("Valid_Traffic_Lights/traffic_light0_00001.jpg")

In [20]:
test_res = label_frame(test)

error: OpenCV(3.4.2) C:\Miniconda3\conda-bld\opencv-suite_1534379934306\work\modules\dnn\src\darknet\darknet_io.cpp:684: error: (-215:Assertion failed) ifile.is_open() in function 'cv::dnn::darknet::ReadDarknetFromWeightsFile'
